In [49]:
from langchain import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Pinecone
import pinecone
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain.llms import CTransformers


In [50]:
PINECONE_API_KEY = "pcsk_2j3Cup_9jrR6tte2C1EEBMaRxYJhii9hkijGx4XgmHSsnSXmC6K89pmRn9uLsaKWMHyK2V"
PINECONE_API_ENV = "us-east-1-aws"

In [51]:
import pypdf


In [52]:
#Extract data from the PDF
def load_pdf(data):
    loader = DirectoryLoader(data,
                    glob="*.pdf",
                    loader_cls=PyPDFLoader)
    
    documents = loader.load()

    return documents

In [53]:
extracted_data = load_pdf(r"C:\Users\premp\OneDrive\Documents\medical_chatbot_llama2\data")  # No slash before 'data'


In [54]:
#Create text chunks
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap = 20)
    text_chunks = text_splitter.split_documents(extracted_data)

    return text_chunks

In [55]:
text_chunks = text_split(extracted_data)
print("length of my chunk:", len(text_chunks))

length of my chunk: 5859


In [56]:
pip install -U sentence-transformers


In [57]:
import os
os.environ["TRANSFORMERS_NO_TF"] = "1"  # Prevent Keras/TensorFlow errors

from langchain.embeddings import HuggingFaceEmbeddings

def download_hugging_face_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embeddings


In [58]:
pip install tf-keras

In [59]:
embeddings = download_hugging_face_embeddings()


In [60]:
embeddings

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False, 'architecture': 'BertModel'})
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
), model_name='sentence-transformers/all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={})

In [61]:
query_result = embeddings.embed_query("Hello world")
print("Length", len(query_result))

Length 384


In [73]:
import pinecone
print(dir(pinecone))

['CollectionDescription', 'Config', 'ConfigBuilder', 'DeleteRequest', 'DeleteResult', 'DescribeIndexStatsRequest', 'DescribeIndexStatsResponse', 'FetchResponse', 'FetchResult', 'ForbiddenException', 'Index', 'IndexDescription', 'InfoResult', 'ListConversionException', 'NotFoundException', 'OpenApiConfiguration', 'PineconeApiAttributeError', 'PineconeApiException', 'PineconeApiKeyError', 'PineconeApiTypeError', 'PineconeApiValueError', 'PineconeConfig', 'PineconeConfigurationError', 'PineconeException', 'PineconeProtocolError', 'ProtobufAny', 'QueryRequest', 'QueryResponse', 'QueryResult', 'QueryVector', 'RpcStatus', 'ScoredVector', 'ServiceException', 'SingleQueryResults', 'SparseValues', 'UnauthorizedException', 'UpdateRequest', 'UpsertRequest', 'UpsertResponse', 'UpsertResult', 'Vector', 'VersionResponse', 'WhoAmIResponse', '__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__path__', '__spec__', '__version__', 'config', 'configure_index', 

In [79]:
pip show pinecone-client

Name: pinecone-client
Version: 2.2.4
Summary: Pinecone client and SDK
Home-page: https://www.pinecone.io/
Author: Pinecone Systems, Inc.
Author-email: support@pinecone.io
License: Proprietary License
Location: c:\users\premp\appdata\roaming\python\python310\site-packages
Requires: dnspython, loguru, numpy, python-dateutil, pyyaml, requests, tqdm, typing-extensions, urllib3
Required-by: 
Note: you may need to restart the kernel to use updated packages.


In [80]:
pip install pinecone

Note: you may need to restart the kernel to use updated packages.


In [78]:
 from pinecone import pinecone as PineconeClient # Renaming to avoid conflict
from langchain_pinecone import Pinecone as PineconeVectorStore # Import the Pinecone vector store from LangChain
pc = PineconeClient(api_key=PINECONE_API_KEY)

index_name = "medical-chatbot"

# Optional: Verify the index exists (good for debugging)
# This step is just for confirmation, the from_texts will connect if it exists.
try:
    # Attempt to describe the index to confirm its existence and status
    index_description = pc.describe_index(index_name)
    print(f"Connected to existing Pinecone index: {index_name}")
    print(f"Index details: {index_description}")
except Exception as e:
    print(f"Error connecting to Pinecone index '{index_name}': {e}")
    print("Please ensure the index exists in your Pinecone account and the API key is correct.")
    # You might want to exit or raise an error here if the index is critical
    exit() # Or raise an exception: raise e

# 2. Creating Embeddings for Each of The Text Chunks & storing
# 'embeddings' should be an instance of an embeddings model (e.g., OpenAIEmbeddings, HuggingFaceEmbeddings)
# The PineconeVectorStore.from_texts method will connect to the existing index
# and then upsert the new embeddings.
docsearch = PineconeVectorStore.from_texts(
    [t.page_content for t in text_chunks], # Assuming text_chunks is a list of LangChain Document objects
    embeddings,
    index_name=index_name
)

print(f"Embeddings created and stored (or updated) in Pinecone index: {index_name}")


ImportError: cannot import name 'PydanticDeprecationWarning' from 'pydantic' (C:\Users\premp\AppData\Roaming\Python\Python310\site-packages\pydantic\__init__.cp310-win_amd64.pyd)

In [68]:
pip show pinecone

Name: pineconeNote: you may need to restart the kernel to use updated packages.

Version: 7.3.0
Summary: Pinecone client and SDK
Home-page: https://www.pinecone.io
Author: Pinecone Systems, Inc.
Author-email: support@pinecone.io
License: Apache-2.0
Location: c:\users\premp\appdata\roaming\python\python310\site-packages
Requires: certifi, pinecone-plugin-assistant, pinecone-plugin-interface, python-dateutil, typing-extensions, urllib3
Required-by: langchain-pinecone


In [4]:
from pinecone import Pinecone

pc = Pinecone(api_key="pcsk_2j3Cup_9jrR6tte2C1EEBMaRxYJhii9hkijGx4XgmHSsnSXmC6K89pmRn9uLsaKWMHyK2V")
index = pc.Index("quickstart")

ImportError: cannot import name 'Pinecone' from 'pinecone' (C:\Users\premp\AppData\Roaming\Python\Python310\site-packages\pinecone\__init__.py)

In [5]:
#Initializing the Pinecone
pinecone.init(api_key=PINECONE_API_KEY )

index_name="medical-chatbot"

#Creating Embeddings for Each of The Text Chunks & storing
docsearch=Pinecone.from_texts([t.page_content for t in text_chunks], embeddings, index_name=index_name)

AttributeError: module 'pinecone.pinecone' has no attribute 'init'

In [6]:
#If we already have an index we can load it like this
docsearch=Pinecone.from_existing_index(index_name, embeddings)

query = "What are Allergies"

docs=docsearch.similarity_search(query, k=3)

print("Result", docs)

NameError: name 'Pinecone' is not defined